# More example

In [5]:
import math
import json
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import Map, Marker, Circle,GeoJSON,Polyline,WidgetControl,FullScreenControl

start_location = (0, 114)
m = Map(center=start_location, zoom=6)

checkbox1 = widgets.Checkbox(value=False, description='Show Marker 1')
checkbox2 = widgets.Checkbox(value=False, description='Show Marker 2')
checkbox3 = widgets.Checkbox(value=False, description='Show Circle')
checkbox4 = widgets.Checkbox(value=False, description='Show GeoJSON')
checkbox5 = widgets.Checkbox(value=False, description='Show Line')
refresh_button = widgets.Button(description='Apply Changes')

button = widgets.Button(description="Open Widgets")
widget_box = widgets.VBox([checkbox1,checkbox2,checkbox3,checkbox4,checkbox5,refresh_button])

def on_button_click(b):
    if widget_box.layout.display == 'none':
        widget_box.layout.display = 'block'
    else:
        widget_box.layout.display = 'none'

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

def destination_point(lat, lon, distance, bearing):
    R = 6371.0
    bearing_rad = math.radians(bearing)

    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)

    lat2_rad = math.asin(math.sin(lat_rad) * math.cos(distance / R) +
                         math.cos(lat_rad) * math.sin(distance / R) * math.cos(bearing_rad))
    lon2_rad = lon_rad + math.atan2(math.sin(bearing_rad) * math.sin(distance / R) * math.cos(lat_rad),
                                    math.cos(distance / R) - math.sin(lat_rad) * math.sin(lat2_rad))

    lat2 = math.degrees(lat2_rad)
    lon2 = math.degrees(lon2_rad)
    return lat2, lon2

def add_marker(m, location):
    marker = Marker(location=location)
    m.add(marker)

def add_circle(m, location):
    circle = Circle(location=location, radius=100000)
    m.add(circle)

def add_geojson(m):
    gdf = gpd.read_file('./demo_widget.geojson')
    geojson_data = json.loads(gdf.to_json())
    geo_json = GeoJSON(data=geojson_data)
    m.add(geo_json)

def add_line(m, start_location):
    end_location = destination_point(start_location[0], start_location[1], 100, 90)
    line = Polyline(locations=[start_location, end_location], color="blue", fill=False)
    m.add(line)

def update_map(*args):
    # Clear existing markers
    for layer in m.layers:
        if isinstance(layer, (Marker, Circle, GeoJSON, Polyline)):
            m.remove_layer(layer)
    # Add markers based on checkbox states
    if checkbox1.value:
        add_marker(m, start_location)
    if checkbox2.value:
        add_marker(m, (2, 114))
    if checkbox3.value:
        add_circle(m, start_location)
    if checkbox4.value:
        add_geojson(m)
    if checkbox5.value:
        add_line(m, start_location)


# Link the button to the update function
refresh_button.on_click(update_map)
button.on_click(on_button_click)
widget_box.layout.display = 'none'  # Initially hide the widget box
button_control = WidgetControl(widget=button, position='topright')
widget_box_control = WidgetControl(widget=widget_box, position='topright')


m.add_control(button_control)
m.add_control(widget_box_control)
m.add(FullScreenControl())

# Display the map and widgets
display(m)

Map(center=[0, 114], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

# Advance pop up

In [3]:
import ipywidgets as widgets
from ipyleaflet import Map, Polygon, Popup
from IPython.display import display

# Create the map
m = Map(center=(0, 114.0), zoom=6)

# Define the polygon
polygon = Polygon(
    locations=[
        (0, 114),
        (1, 114),
        (1, 115),
        (0, 115)
    ],
    color="green",
    fill_color="green"
)

# Create an image carousel
images = [
    widgets.Image(value=open('./image/image_1.jpg', 'rb').read(), format='png'),
    widgets.Image(value=open('./image/image_2.jpg', 'rb').read(), format='png'),
    widgets.Image(value=open('./image/image_3.jpg', 'rb').read(), format='png')
]
carousel = widgets.Box(children=images)

# Create a title widget
title = widgets.HTML("<h2>Example Image Carousel</h2>")

# Combine title and carousel in a VBox
popup_content = widgets.VBox([title, carousel])

# Function to create a popup
def create_popup():
    return Popup(
        location=(0.5, 114.5),
        child=popup_content,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
        keep_in_view=True,
        min_width = 300
    )

# Add the popup to the polygon on click
def on_click(**kwargs):
    popup = create_popup()
    m.add_layer(popup)

polygon.on_click(on_click)

# Add the polygon to the map
m.add_layer(polygon)

display(m)


Map(center=[0, 114.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

# Advance popup using geodataframe

**(jika error bisa coba check di jupyterLab untuk melihat error log jika tidak muncul)**

In [4]:
import geopandas as gpd
import shapely.geometry as geo
from ipyleaflet import Map, Polygon, Popup
import ipywidgets as widgets
from IPython.display import display

# Example GeoDataFrame with polygons
gdf = gpd.GeoDataFrame({
    'geometry': [
        geo.Polygon([(114, 0), (114, 1), (115, 1), (115, 0)]),
        geo.Polygon([(114, 2), (114, 3), (115, 3), (115, 2)]),
    ],
    'crs': "EPSG:4326"
})

# Create the map
m = Map(center=(0, 114), zoom=6)

# Function to create a popup
def create_popup(lat, lon):
    # Create an image carousel or any other content you want
    images = [
        widgets.Image(value=open('./image/image_1.jpg', 'rb').read(), format='png'),
        widgets.Image(value=open('./image/image_2.jpg', 'rb').read(), format='png'),
        widgets.Image(value=open('./image/image_3.jpg', 'rb').read(), format='png')
    ]
    carousel = widgets.Box(children=images)
    title = widgets.HTML(f"<h3>Polygon at Lat={round(lat, 2)} | Lon={round(lon, 2)}</h3>")
    popup_content = widgets.VBox([title, carousel])
    
    return Popup(
        location=(lat, lon),
        child=popup_content,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
        keep_in_view=True,
        min_width=300
    )

def on_click_polygon(**kwargs):
    coordinates = kwargs.get('coordinates', None)
    if coordinates and len(coordinates) == 2 and all(isinstance(coord, (int, float)) for coord in coordinates):
        popup = create_popup(lat=coordinates[0], lon=coordinates[1])
        m.add(popup)

# Add polygons and popups to the map
for idx, row in gdf.iterrows():
    polygon = Polygon(locations=[list(coord[::-1]) for coord in row.geometry.exterior.coords])
    polygon.on_click(on_click_polygon)
    m.add(polygon)

m.layout.height = '80vh'
display(m)


Map(center=[0, 114], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…